## Imports

In [1]:
from itertools import chain

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default

import pandas as pd
import numpy as np
from random import choice

## Functions

In [2]:



def give_hint(answer, skip_list):
  """
  This function gives a hint based on the answer.
  All characters that aren't letters are shown in the hint.
  If the answer starts with a word for skip_list it is shown in the hint.
  The first letter is (after the word from skip_list) shown in the hint.
  All other letters are shown as dots.

  Examples:
  answer = "solution" --> hint = "s......."
  answer = "Hello everyone!" --> hint = "H.... ........!"
  answer = "!Hola!" --> hint = '!H...!"
  answer = "la tortuga" --> hint = "la t......"
  """

  hint = ""
  hint_index = 0 # the index of the character that should be given as hint

  # words from skiplist are shown in the hint
  for word in skip_list:
    if answer.lower().startswith(word.strip() + " "): # check if answer starts with this word
      hint += answer[:len(word)+1]                    # add this word to the hint
      hint_index = len(word)+1
      break # no need to check the other words in skip_list

  # show the first letter
  for ch in answer[hint_index:]:
    hint_index += 1
    if ( ch >= 'a' and ch <= 'z' ) or ( ch >= 'A' and ch <= 'Z'):
      hint += ch  # add the letter that gives the hint
      break
    else:
      hint += ch # add the ch that isn't a letter

  # for all other characters; dots for letters, other characters are shown
  for ch in answer[hint_index:]:
    if ( ch >= 'a' and ch <= 'z' ) or ( ch >= 'A' and ch <= 'Z'):
      hint += "."
    else:
      hint += ch

  return hint



def clean_string(string1):
  """
  This function cleans the string.
  It removes unnecessary spaces.
  It removes text in between the breakets () since that's
  extra information the user doesn't need to type in during the quiz.
  """

  # Removing breackets + text in between them
  start = -1
  while True:
    start = string1.find("(")
    if start < 0:
      break
    finish = string1.find(")")
    if finish < 0:
      break
    string1 = string1[:start] + string1[finish+1:] # removing the text in between breakets

  # remove unnessary spaces on beginning and end and remove double spaces
  string1 = string1.strip().replace("  ", " ")

  return string1



def get_language_order(words_df):
  """
  You can determine from which language to which language you want to learn.
  Example: you get to see the words in English and have to give the answer in Spanish.

  from_language --> the language the quiz shows you
  to_language --> the language in which you have to answer
  """

  language1 = words_df.columns[0]
  language2 = words_df.columns[1]

  language_options = "" # lege string

  while language_options != '1' and language_options != '2':
    language_options = input("Choose the dirextion you want to learn the language? \n 1. {language1} -> {language2} \n 2. {language2} -> {language1} \n Type 1 or 2. \n".format(language1=language1,language2=language2  ))

    if language_options == '1':
      from_language = language1
      to_language = language2
    elif language_options == '2':
      from_language = language2
      to_language = language1
    else:
      print("You didn't answer correctly! Please try again!")

    print(f"Great! We will quiz words from {from_language} to {to_language}.\n")

  return(from_language, to_language)


def select_quiz_type():
  quiz_number = -1
  quiz_number = input("""What kind of quiz would you like to do?
    1. Flascards - Come up with the translation in your head.
    2. Quiz - Type in the answer. Each word is asked once.
    3. Repeated Quiz - Type in the answer. Repeated until correct.
    4. Multiple choice - Repeated until correct.
  Type in which quiz type you want (1,2,3,4): """)
  # [add quiz_number 5 later]

  while quiz_number not in ('1','2','3', '4'):
    quiz_number = input("This input is incorrect. Please type in one of the following numbers (1, 2, 3, 4): ")
  quiz_number = int(quiz_number)

  # Select action for user
  if quiz_number == 1:
    action_string = "Type 'h' if you want a hint. \n Press ENTER when you have the word in your head.\n"
  elif quiz_number == 2 or quiz_number ==3:
    action_string = "Type 'h' if you want a hint. \n Type in your answer: \n"
  elif quiz_number == 4:
    action_string = "Type the number with the correct answer: \n"
  # [add 5 later]

  return quiz_number, action_string


## Main

In [ ]:
#### 4 QUIZ OPTIONS

## Get words dataframe

creds, _ = default()
gc = gspread.authorize(creds)

file_name = input("What is the name of the file you want to use? \n For example: 'WRTS input' \n")

worksheet = gc.open(file_name).sheet1
rows = worksheet.get_all_values() # get_all_values gives a list of rows.

words_df = pd.DataFrame.from_records(rows)
words_df.rename(columns=words_df.iloc[0], inplace=True) # make columns names equal to the names on the first row
words_df.drop(words_df.index[0], inplace=True) # drop the values in row 0
print(words_df)



## Get skip_list

worksheet1 = gc.open('skip words').sheet1
rows = worksheet1.get_all_values()            # get_all_values gives a list of rows
skip_list = list(chain.from_iterable(rows))   # list with all words
print(skip_list)



## Give up language order

from_language, to_language = get_language_order(words_df)


## Decide type of quiz

quiz_number, action_string = select_quiz_type()


## START QUIZ

nr_words_tested = 0
nr_words_correct = 0
test_file = words_df.copy() # need a copy otherwise a change in testingbestand will mean a change in wordsfile

while len(test_file) > 0:

  # Select the word to be translated
  row_index = choice(list(test_file.index)) # randomly choose row index from test_file
  translate_word = test_file[from_language][row_index] # [column_name][row_index]
  correct_answer = test_file[to_language][row_index] # [column_name][row_index]
  nr_words_tested += 1

  if quiz_number == 4:
    random_list = np.random.choice(words_df.drop([row_index]).index, 3, replace=False) # randomly choice 3 indicex (excluding that of the correct answer)
    random_list = np.append(random_list, row_index) # add the index of the correct answer
    np.random.shuffle(random_list) # shuffle row_indices

    # update action string
    action_string = f"""The options are:
    1. {words_df[to_language][random_list[0]]}
    2. {words_df[to_language][random_list[1]]}
    3. {words_df[to_language][random_list[2]]}
    4. {words_df[to_language][random_list[3]]}
    Type the number with the correct answer: \n"""

  # Ask user for input
  user_input = input(f"""\n Can you translate '{translate_word}' to {to_language}?
    {action_string}""")


  if quiz_number == 4:
    while user_input not in ("1", "2", "3", "4"):
      user_input = input("This input is incorrect. Please fill out one of the following integers: 1, 2, 3, 4.\n")
    user_input = int(user_input)
    user_input = words_df[to_language][random_list[user_input-1]] # find word corresponding to this input

  # Give hint if asked
  if user_input == 'h':
    hint = give_hint(correct_answer, skip_list)
    user_input = input(f"The hint is: '{hint}'. Please try again: \n")

  # check if correct answer is typed in
  if user_input == clean_string(correct_answer):
    print("This answer is CORRECT!\n")
    nr_words_correct += 1
  else:
    if user_input != clean_string(correct_answer) and quiz_number != 1:
      print("This answer is WRONG!")
    print(f"The correct answer is '{correct_answer}'.\n")

  # remove this row if answer is correct OR we only quiz each word once
  if quiz_number ==1 or quiz_number == 2 or user_input == clean_string(correct_answer):
    test_file.drop([row_index], inplace=True)

print("YOU'VE FINISHED THE QUIZ!")
if quiz_number in (2,3,4):
  grade = nr_words_correct/nr_words_tested * 10
  print(f"You're grade is: {grade:2.1f}")


What is the name of the file you want to use? 
 For example: 'WRTS input' 
dieren - nederlands/spaans
          Dutch     Spanish
1        De koe     La vaca
2        De uil     El búho
3       De hond    El perro
4        De kat     El gato
5        De vis      El pez
6  De schildpad  La tortuga
['to', 'a', 'an', 'la', 'el', 'un', 'una', 'een', 'het', 'de']
Choose the dirextion you want to learn the language? 
 1. Dutch -> Spanish 
 2. Spanish -> Dutch 
 Type 1 or 2. 
1
Great! We will quiz words from Dutch to Spanish.



## Testing functions

In [ ]:
# Test give_hint function

print(give_hint("!actie", skip_list))
print(give_hint("To add", skip_list))
print(give_hint("Hello?", skip_list))
print(give_hint("!Hola", skip_list))
print(give_hint("Hola, mucho gusto!", skip_list))


# Test clean_string function

print(clean_string("Hello  You"))
print(clean_string(' to know (people, places) (**more information**)'))
print(clean_string("  to know "))


# Test get_language_order function

df_words = pd.DataFrame([['el gato', 'the cat'], [ 'el perro', 'the dog']],columns=['Spanish', 'English'])
get_language_order(df_words)

!a....
To a..
H....?
!H...
H..., ..... .....!
Hello You
to know
to know
Choose the dirextion you want to learn the language? 
 1. Spanish -> English 
 2. English -> Spanish 
 Type 1 or 2. 
1
Great! We will quiz words from Spanish to English.


('Spanish', 'English', 0, 1)